In [1]:
REWARD_FNS = {}

### Fonctions de récompense pour Walker2d-v5

Dans ce projet, nous voulons étudier l'effet de différentes fonctions de récompense pour l'environnement `Walker2d-v5` (Gymnasium / MuJoCo). Dans la version de base, la récompense à chaque pas est la somme de trois termes :

- **healthy_reward** : bonus de survie lorsque le robot reste dans une zone considérée comme "saine" ;
- **forward_reward** : récompense proportionnelle à la vitesse vers l'avant (déplacement en x par unité de temps) ;
- **ctrl_cost** : coût quadratique sur les actions (pénalise les torques trop élevés).

La récompense totale est donc :

> reward = healthy_reward + forward_reward − ctrl_cost

et `info` contient les termes individuels sous les clés  
`"reward_forward"`, `"reward_ctrl"`, `"reward_survive"`.  
(Cf. la documentation officielle de `Walker2d-v5`.)

Dans le cadre du *reward shaping*, on modifie cette fonction de récompense pour guider l'apprentissage, par exemple en ajoutant des termes de posture, de vitesse cible ou de lissage des actions. Ce type de modification est étudié de manière théorique dans l'article classique de Ng, Harada et Russell, *"Policy Invariance Under Reward Transformations: Theory and Application to Reward Shaping"* (ICML 1999), qui montre dans quels cas certaines transformations de la récompense conservent la politique optimale. :contentReference[oaicite:1]{index=1}

Le code ci-dessous implémente plusieurs variantes de récompense pour `Walker2d-v5` sous forme d'un wrapper Gymnasium, de façon à pouvoir sélectionner facilement une fonction de récompense et entraîner un agent avec celle-ci.


### Fonction de récompense 1 : vitesse – énergie – survie

Dans l'environnement `Walker2d-v5` (Gymnasium / MuJoCo), la récompense par défaut est décomposée en trois termes :

- `reward_forward` : récompense pour la vitesse vers l'avant ;
- `reward_ctrl` : coût de contrôle (négatif), proportionnel au carré des actions ;
- `reward_survive` : bonus de survie tant que le robot reste dans un état "sain".

Voir la documentation de `Walker2d-v5` qui précise cette décomposition de la récompense dans le dictionnaire `info`.   

Pour notre première fonction de récompense, nous définissons une combinaison linéaire de ces trois termes :

$
r_t = w_{\text{forward}} \cdot \text{reward\_forward}
    + w_{\text{ctrl}} \cdot \text{reward\_ctrl}
    + w_{\text{survive}} \cdot \text{reward\_survive}
$

où :

- $w_{\text{forward}}$ contrôle l'importance de la vitesse ;
- $w_{\text{ctrl}}$ contrôle l'importance de l'énergie consommée (comme `reward_ctrl` est négatif, un poids positif signifie "on pénalise l'énergie") ;
- $w_{\text{survive}}$ contrôle l'importance du bonus de survie.

En faisant varier ces poids, on peut étudier le compromis entre **vitesse**, **consommation d'énergie** et **stabilité** (survie) du marcheur, ce qui est particulièrement intéressant dans un contexte de perturbations (bruit d'observation, randomisation des conditions initiales).

In [2]:
from typing import Dict, Any

def reward_speed_energy(
    info: Dict[str, Any],
    w_forward: float = 1.0,
    w_ctrl: float = 1.0,
    w_survive: float = 1.0,
) -> float:
    """
    Fonction de récompense 1 : combinaison vitesse / énergie / survie
    pour Walker2d-v5.

    Paramètres
    ----------
    info : dict
        Le dictionnaire `info` renvoyé par env.step(action).
        On suppose qu'il contient les clés :
        - "reward_forward"
        - "reward_ctrl"
        - "reward_survive"
    w_forward : float
        Poids de la récompense de vitesse (reward_forward).
    w_ctrl : float
        Poids du coût de contrôle (reward_ctrl).
        Attention : reward_ctrl est déjà négatif dans Walker2d.
        Un w_ctrl > 0 correspond donc à une pénalisation de l'énergie.
    w_survive : float
        Poids du bonus de survie (reward_survive).

    Retour
    ------
    float
        La récompense scalaire r_t.
    """
    forward = float(info.get("reward_forward", 0.0))
    ctrl = float(info.get("reward_ctrl", 0.0))         # déjà négatif
    survive = float(info.get("reward_survive", 0.0))

    reward = (
        w_forward * forward
        + w_ctrl * ctrl
        + w_survive * survive
    )
    return reward

# obs, base_reward, terminated, truncated, info = env.step(action)
# new_reward = reward_speed_energy(
#     info,
#     w_forward=1.0,
#     w_ctrl=1.0,
#     w_survive=1.0,
# )


In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import SAC

# --- 1) Wrapper qui remplace la récompense par reward_speed_energy ---

class SpeedEnergyRewardWrapper(gym.Wrapper):
    def __init__(self, env, w_forward=1.0, w_ctrl=1.0, w_survive=1.0):
        super().__init__(env)
        self.w_forward = w_forward
        self.w_ctrl = w_ctrl
        self.w_survive = w_survive

    def step(self, action):
        # on appelle l'env "normal"
        obs, base_reward, terminated, truncated, info = self.env.step(action)

        # on calcule NOTRE récompense à partir de info
        new_reward = reward_speed_energy(
            info,
            w_forward=self.w_forward,
            w_ctrl=self.w_ctrl,
            w_survive=self.w_survive,
        )

        # on renvoie obs, new_reward (et pas base_reward)
        return obs, new_reward, terminated, truncated, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


# --- 2) Création de l'environnement + enregistrement vidéo ---

video_folder = "./videos_speed_energy"

# IMPORTANT : render_mode="rgb_array" pour pouvoir faire une vidéo
env_base = gym.make("Walker2d-v5", render_mode="rgb_array")

# on applique notre wrapper de récompense
env_wrapped = SpeedEnergyRewardWrapper(
    env_base,
    w_forward=1.0,
    w_ctrl=1.0,
    w_survive=1.0,
)

# on ajoute le wrapper vidéo
env = RecordVideo(
    env_wrapped,
    video_folder=video_folder,
    name_prefix="walker2d-speed_energy",
    episode_trigger=lambda ep_id: True,  # filme tous les épisodes
    video_length=0,                       # 0 = filme l'épisode complet
)


# --- 3) Entraînement rapide avec SAC ---

model = SAC("MlpPolicy", env, verbose=1)

# nombre de pas tout petit pour tester (à augmenter plus tard)
model.learn(total_timesteps=5_000)


# --- 4) On filme un épisode avec le modèle entraîné ---

obs, info = env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

env.close()
print(f"Épisode terminé. Vidéo enregistrée dans : {video_folder}")


### Fonction de récompense 2 : vitesse cible

L'idée de cette récompense est de ne plus dire "plus vite = mieux", mais plutôt "le robot doit marcher à **une vitesse cible** $v^*$", par exemple $v^* \in \{1.0, 2.0\}$ m/s.

On note :

- $v_x$ : la vitesse horizontale du torse (dans `Walker2d-v5`, c'est `obs[8]`, la vitesse en x du torse). 
- $a_t$ : le vecteur d'actions (torques) au temps $t$,
- `reward_survive` : le bonus de survie fourni par l'environnement.

La récompense est définie par :

$
r_t = - \alpha\, |v_x - v^*|
      - \beta \, \|a_t\|^2
      + w_{\text{survive}} \cdot \text{reward\_survive}
$

où :

- $\alpha$ contrôle l'importance d'être proche de la vitesse cible $v^*$,
- $\beta$ contrôle la pénalisation de l'énergie (norme au carré des actions),
- $w_{\text{survive}}$ ajuste l'importance du terme de survie.

#### Intérêt sous perturbations

Quand on ajoute du bruit (sur les observations ou les conditions initiales), maximiser simplement la vitesse pousse souvent l'agent à **courir de plus en plus vite**, avec des mouvements nerveux et peu stables.

Avec cette fonction de récompense à **vitesse cible** :

- l'agent est encouragé à maintenir une **vitesse régulière** proche de $v^*$,
- on peut mesurer :
  - la variance de la vitesse $v_x$ au cours d'un épisode,
  - la variance des actions,
  - l'énergie consommée ($\sum_t \|a_t\|^2$),

ce qui permet de comparer différentes politiques en termes de **stabilité** et de **robustesse** face aux perturbations.


In [5]:
from typing import Dict, Any
import numpy as np

def reward_target_speed(
    obs,
    action,
    info: Dict[str, Any],
    v_target: float = 1.5,
    alpha: float = 1.0,
    beta: float = 1e-3,
    w_survive: float = 1.0,
) -> float:
    """
    Récompense 2 : vitesse cible + coût d'énergie + survie.

        r_t = - alpha * |v_x - v_target|
              - beta  * ||a_t||^2
              + w_survive * reward_survive

    - v_x : vitesse en x du torse (obs[8] dans Walker2d-v5)
    - a_t : action au temps t
    """

    # Vitesse horizontale du torse.
    # D'après la doc Walker2d-v5, obs[8] = vitesse en x du torse.
    vx = float(obs[8])

    # Norme au carré de l'action (énergie "dépensée")
    energy = float(np.sum(np.square(action)))

    # Terme de survie fourni par l'env (comme pour reward_speed_energy)
    survive = float(info.get("reward_survive", 0.0))

    # Terme de vitesse : on veut que v_x soit proche de v_target
    speed_term = -alpha * abs(vx - v_target)

    # Terme d'énergie (pénalisation)
    energy_term = -beta * energy

    # Récompense totale
    reward = speed_term + energy_term + w_survive * survive
    return reward


In [6]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import SAC

# --- Wrapper pour la récompense "vitesse cible" ---

class TargetSpeedRewardWrapper(gym.Wrapper):
    def __init__(self, env, v_target=1.5, alpha=1.0, beta=1e-3, w_survive=1.0):
        super().__init__(env)
        self.v_target = v_target
        self.alpha = alpha
        self.beta = beta
        self.w_survive = w_survive

    def step(self, action):
        # Env de base
        obs, base_reward, terminated, truncated, info = self.env.step(action)

        # Notre nouvelle récompense
        new_reward = reward_target_speed(
            obs=obs,
            action=action,
            info=info,
            v_target=self.v_target,
            alpha=self.alpha,
            beta=self.beta,
            w_survive=self.w_survive,
        )

        return obs, new_reward, terminated, truncated, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


# --- Création de l'env + enregistrement vidéo ---

video_folder = "./videos_target_speed"

env_base = gym.make("Walker2d-v5", render_mode="rgb_array")

env_wrapped = TargetSpeedRewardWrapper(
    env_base,
    v_target=1.5,   # vitesse cible en m/s (tu peux tester 1.0, 2.0, etc.)
    alpha=1.0,
    beta=1e-3,
    w_survive=1.0,
)

env = RecordVideo(
    env_wrapped,
    video_folder=video_folder,
    name_prefix="walker2d-target_speed",
    episode_trigger=lambda ep_id: True,
    video_length=0,
)

# --- Entraînement rapide avec SAC (juste pour tester) ---

model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)   # à augmenter plus tard

# --- On filme un épisode avec le modèle entraîné ---

obs, info = env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

env.close()
print(f"Épisode terminé. Vidéo enregistrée dans : {video_folder}")

c:\Users\arthu\miniconda3\envs\walker2d\lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at c:\Users\arthu\OneDrive\Documents\UTC\Poly mtl\RL\Projet\videos_target_speed folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27.2     |
|    ep_rew_mean     | -36.5    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 43       |
|    time_elapsed    | 2        |
|    total_timesteps | 109      |
| train/             |          |
|    actor_loss      | -5.27    |
|    critic_loss     | 1.89     |
|    ent_coef        | 0.998    |
|    ent_coef_loss   | -0.0208  |
|    learning_rate   | 0.0003   |
|    n_updates       | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27.8     |
|    ep_rew_mean     | -37.6    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 34       |
|    time_elapsed    | 6        |
|    total_timesteps | 222      |
| train/             |

### Fonction de récompense 3 : posture stable

Cette récompense vise à encourager non seulement le déplacement vers l'avant, mais aussi une **posture stable** : marcher "debout et droit".

Dans `Walker2d-v5` :

- `info["reward_forward"]` : terme de vitesse vers l'avant,
- `info["reward_ctrl"]` : coût de contrôle (négatif, pénalise les grandes actions),
- `info["reward_survive"]` : bonus de survie lorsque le robot reste dans une zone "saine",
- `obs[0]` : hauteur du torse,
- `obs[1]` : angle du torse (0 = vertical). 
La récompense est définie comme :

$
r_t = w_{\text{forward}} \cdot \text{reward\_forward}
    + w_{\text{ctrl}} \cdot \text{reward\_ctrl}
    + w_{\text{survive}} \cdot \text{reward\_survive}
    - w_h \, \max(0, h_{\text{target}} - h_t)^2
    - w_{\text{angle}} \, \theta_t^2
$

où :

- $h_t$ est la hauteur du torse au temps $t$,
- $\theta_t$ est l'angle du torse (0 = droit),
- $h_{\text{target}}$ est une hauteur cible (par ex. 1.25),
- $(w_h$ contrôle l'importance d'être suffisamment haut,
- $w_{\text{angle}}$ contrôle l'importance d'être droit.

#### Enjeux et intérêt avec bruit / resets

Avec du bruit d'observation et des conditions initiales aléatoires, le walker a tendance à :

- se pencher beaucoup vers l'avant ou l'arrière,
- s'affaisser (baisser la hauteur du torse) avant de tomber.

Cette récompense "posture stable" :

- pénalise progressivement les **postures dangereuses** (trop penché, trop bas),
- fournit un signal de récompense **avant la chute**,
- encourage des gaits plus **stables** et donc souvent plus **robustes** aux perturbations.

On peut comparer cette récompense à d'autres en mesurant :
- la hauteur moyenne du torse,
- la variance de l'angle du torse,
- le nombre de chutes / terminaisons par épisode.


In [13]:
from typing import Dict, Any

def reward_posture_stability(
    obs,
    action,
    info: Dict[str, Any],
    h_target: float = 1.25,
    w_forward: float = 1.0,
    w_ctrl: float = 1.0,
    w_survive: float = 1.0,
    w_h: float = 1.0,
    w_angle: float = 1.0,
) -> float:
    """
    Récompense 3 : posture stable

    Idée :
      - garder les termes classiques (vitesse, coût de contrôle, survie)
      - ajouter des termes qui encouragent une posture "debout et droite"

    r_t = w_forward * reward_forward
        + w_ctrl    * reward_ctrl
        + w_survive * reward_survive
        + height_term
        + angle_term

    où :
      - height_term pénalise si la hauteur du torse est en dessous d'une cible h_target
      - angle_term pénalise si l'angle du torse s'éloigne de 0
    """

    # --- 1) Termes classiques de Walker2d ---

    forward = float(info.get("reward_forward", 0.0))
    ctrl = float(info.get("reward_ctrl", 0.0))         # déjà négatif
    survive = float(info.get("reward_survive", 0.0))

    base_terms = (
        w_forward * forward
        + w_ctrl * ctrl
        + w_survive * survive
    )

    # --- 2) Termes de posture ---

    # D'après la doc Walker2d-v5 :
    #   obs[0] = hauteur du torse
    #   obs[1] = angle du torse (0 = droit).  :contentReference[oaicite:0]{index=0}
    h = float(obs[0])
    angle = float(obs[1])

    # pénalité si le torse est plus bas que h_target (quadratique)
    height_penalty = -w_h * max(0.0, h_target - h) ** 2

    # pénalité quadratique sur l'angle (on veut angle ≈ 0)
    angle_penalty = -w_angle * angle ** 2

    reward = base_terms + height_penalty + angle_penalty
    return reward


In [14]:
import gymnasium as gym
from stable_baselines3 import SAC

# --- ENV D'ENTRAÎNEMENT (pas de vidéo ici) ---

train_env_base = gym.make("Walker2d-v5")  # PAS de render_mode pour l'entraînement

train_env = PostureStabilityRewardWrapper(
    train_env_base,
    h_target=1.25,
    w_forward=1.0,
    w_ctrl=1.0,
    w_survive=1.0,
    w_h=5.0,
    w_angle=1.0,
)

model = SAC("MlpPolicy", train_env, verbose=1)
model.learn(total_timesteps=10_000)   # à augmenter plus tard

# (optionnel) sauver le modèle
model.save("sac_walker2d_posture_stable")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 18.5     |
|    ep_rew_mean     | -5.22    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 2286     |
|    time_elapsed    | 0        |
|    total_timesteps | 74       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.6     |
|    ep_rew_mean     | -5.74    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 229      |
|    time_elapsed    | 0        |
|    total_timesteps | 157      |
| train/             |          |
|    actor_loss      | -7.55    |
|    critic_loss     | 3.52     |
|    ent_coef        | 0.984    |
|    ent_coef_loss   | -0.167   |
|    learning_rate   | 0.0003   |
|    n_updates       | 56       |
----------------------

In [15]:
from gymnasium.wrappers import RecordVideo

# --- ENV D'ÉVALUATION AVEC VIDÉO ---

video_folder = "./videos_posture_stable"

eval_env_base = gym.make("Walker2d-v5", render_mode="rgb_array")

eval_env_wrapped = PostureStabilityRewardWrapper(
    eval_env_base,
    h_target=1.25,
    w_forward=1.0,
    w_ctrl=1.0,
    w_survive=1.0,
    w_h=5.0,
    w_angle=1.0,
)

eval_env = RecordVideo(
    eval_env_wrapped,
    video_folder=video_folder,
    name_prefix="walker2d-posture_stable",
    episode_trigger=lambda ep_id: True,  # on filme le premier épisode
    video_length=0,                       # 0 = épisode complet
)

# si tu as sauvegardé le modèle :
# model = SAC.load("sac_walker2d_posture_stable", env=eval_env)

obs, info = eval_env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)

eval_env.close()
print(f"Vidéo enregistrée dans : {video_folder}")


c:\Users\arthu\miniconda3\envs\walker2d\lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at c:\Users\arthu\OneDrive\Documents\UTC\Poly mtl\RL\Projet\videos_posture_stable folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Vidéo enregistrée dans : ./videos_posture_stable


### Fonction de récompense 4 : actions lisses

Cette récompense cherche à éviter les politiques qui réagissent de manière **trop nerveuse** au bruit (observations bruitées, resets aléatoires, etc.).  
L'idée est de pénaliser les **changements brusques d'actions** entre deux pas de temps :

$
r_t = \text{base\_reward}_t - \lambda \, \lVert a_t - a_{t-1} \rVert^2,
$

où :

- $\text{base\_reward}_t$ est la récompense originale de l'environnement `Walker2d-v5` (vitesse + survie − coût de contrôle)   
- $a_t$ est l'action au temps t,
- $a_{t-1}$ est l'action au temps $t-1$,
- $\lambda$ (noté `lambda_smooth` dans le code) contrôle la force de la pénalisation des changements d'action.

#### Enjeux et intérêt avec du bruit

Avec du bruit d'observation, les algorithmes RL ont tendance à :

- réagir fortement à de petites fluctuations,
- produire des actions qui oscillent très vite,
- avoir des gaits moins stables et parfois plus coûteux en énergie.

La récompense "actions lisses" encourage :

- des **commandes plus régulières** (moins d'oscillations haute fréquence),
- des gaits souvent plus **stables** et plus **robustes** au bruit,
- une politique plus "conservatrice" dans ses changements d'actions.

On peut comparer différentes valeurs de $\lambda$ (par ex. `1e-3`, `1e-2`, `5e-2`) et mesurer :
- la variance des actions,
- la consommation d'énergie,
- la robustesse aux perturbations.


In [16]:
from typing import Dict, Any
import numpy as np

def reward_smooth_actions(
    obs,
    action,
    info: Dict[str, Any],
    base_reward: float,
    prev_action,
    lambda_smooth: float = 1e-2,
) -> float:
    """
    Récompense 4 : actions lisses (anti-réaction nerveuse au bruit)

    r_t = base_reward_t - lambda_smooth * ||a_t - a_{t-1}||^2

    - base_reward_t : récompense originale de l'environnement Walker2d-v5
    - a_t           : action actuelle
    - a_{t-1}       : action précédente
    """
    # Si on n'a pas encore d'action précédente (premier step), on ne pénalise pas
    if prev_action is None:
        return float(base_reward)

    # Différence entre action actuelle et précédente
    delta = action - prev_action

    # Norme au carré de la différence (grands changements d'actions = pénalisés)
    smooth_penalty = lambda_smooth * float(np.sum(delta ** 2))

    new_reward = float(base_reward) - smooth_penalty
    return new_reward


In [17]:
import gymnasium as gym

class SmoothActionsRewardWrapper(gym.Wrapper):
    def __init__(self, env, lambda_smooth: float = 1e-2):
        super().__init__(env)
        self.lambda_smooth = lambda_smooth
        self.prev_action = None

    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        # Au reset, on n'a pas encore d'action précédente
        self.prev_action = None
        return obs, info

    def step(self, action):
        # Appel de l'env de base
        obs, base_reward, terminated, truncated, info = self.env.step(action)

        # Calcul de notre récompense "actions lisses"
        new_reward = reward_smooth_actions(
            obs=obs,
            action=action,
            info=info,
            base_reward=base_reward,
            prev_action=self.prev_action,
            lambda_smooth=self.lambda_smooth,
        )

        # Mise à jour de l'action précédente
        self.prev_action = np.array(action, copy=True)

        return obs, new_reward, terminated, truncated, info


In [18]:
from stable_baselines3 import SAC

# --- ENV D'ENTRAÎNEMENT (sans vidéo) ---

train_env_base = gym.make("Walker2d-v5")  # pas de render_mode ici
train_env = SmoothActionsRewardWrapper(
    train_env_base,
    lambda_smooth=1e-2,  # tu peux tester différentes valeurs
)

model_smooth = SAC("MlpPolicy", train_env, verbose=1)
model_smooth.learn(total_timesteps=10_000)   # à augmenter plus tard

# optionnel : sauver le modèle
# model_smooth.save("sac_walker2d_smooth_actions")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.2     |
|    ep_rew_mean     | -2.6     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 1648     |
|    time_elapsed    | 0        |
|    total_timesteps | 77       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.1     |
|    ep_rew_mean     | -2.29    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 234      |
|    time_elapsed    | 0        |
|    total_timesteps | 153      |
| train/             |          |
|    actor_loss      | -7.68    |
|    critic_loss     | 2.6      |
|    ent_coef        | 0.985    |
|    ent_coef_loss   | -0.156   |
|    learning_rate   | 0.0003   |
|    n_updates       | 52       |
----------------------

In [19]:
from gymnasium.wrappers import RecordVideo

video_folder = "./videos_smooth_actions"

# --- ENV D'ÉVALUATION AVEC VIDÉO ---

eval_env_base = gym.make("Walker2d-v5", render_mode="rgb_array")
eval_env_wrapped = SmoothActionsRewardWrapper(
    eval_env_base,
    lambda_smooth=1e-2,
)

eval_env = RecordVideo(
    eval_env_wrapped,
    video_folder=video_folder,
    name_prefix="walker2d-smooth_actions",
    episode_trigger=lambda ep_id: True,  # on filme le premier épisode
    video_length=0,                       # épisode complet
)

# Si tu avais sauvegardé : model_smooth = SAC.load("sac_walker2d_smooth_actions", env=eval_env)

obs, info = eval_env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action, _ = model_smooth.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)

eval_env.close()
print(f"Vidéo enregistrée dans : {video_folder}")


Vidéo enregistrée dans : ./videos_smooth_actions


### Fonction de récompense 5 : anti-chute progressive

L'objectif de cette récompense est de fournir un **signal de danger progressif** quand le walker se rapproche de la chute, au lieu d'avoir seulement une grosse pénalité quand l'épisode se termine.

Dans `Walker2d-v5`, l'observation contient notamment :  
- `obs[0]` : la hauteur du torse,  
- `obs[1]` : l'angle du torse (0 = vertical).  

La récompense de base de l'environnement (vitesse vers l'avant, coût de contrôle, survie) est notée $\text{base\_reward}_t$.  
On définit alors :

$
r_t = \text{base\_reward}_t
      - w_h \, \max(0, h_{\text{crit}} - h_t)^2
      - w_{\text{angle}} \, \max(0, |\theta_t| - \theta_{\text{crit}})^2
$

où :

- $h_t$ est la hauteur du torse au temps t,
- $\theta_t$ est l'angle du torse (0 = droit),
- $h_{\text{crit}}$ est une hauteur "critique" en dessous de laquelle le robot est proche de se coucher,
- $theta_{\text{crit}}$ est un angle limite au-delà duquel le robot est trop penché,
- $w_h$ et $w_{\text{angle}}$ contrôlent l'importance de ces pénalités.

#### Intérêt du shaping "anti-chute progressive"

Avec cette forme :

- les états **dangereux** (trop bas, trop penchés) sont pénalisés **avant** la chute,
- l'agent peut apprendre à **se redresser** et **se rattraper** au lieu de simplement tomber,
- en présence de bruit (observations bruitées, resets aléatoires), cela aide à apprendre des comportements plus **robustes** :
  - moins de chutes,
  - plus de temps passé dans des états "sûrs".


In [20]:
from typing import Dict, Any
import numpy as np

def reward_anti_fall_progressive(
    obs,
    action,
    info: Dict[str, Any],
    base_reward: float,
    h_crit: float = 0.9,
    angle_crit: float = 0.5,
    w_h: float = 5.0,
    w_angle: float = 1.0,
) -> float:
    """
    Récompense 5 : anti-chute progressive (shaping autour des états dangereux)

    Idée :
      - on part de la récompense de base de l'environnement (base_reward)
      - on ajoute des pénalités "douces" quand le walker se rapproche de la chute :
          * torse trop bas  (h < h_crit)
          * torse trop penché (|angle| > angle_crit)

    Formule :
        r_t = base_reward
              - w_h     * max(0, h_crit - h_t)^2
              - w_angle * max(0, |angle_t| - angle_crit)^2

    où :
      - h_t      : hauteur du torse au temps t
      - angle_t  : angle du torse (0 = vertical)
      - h_crit   : hauteur critique en dessous de laquelle on "approche" de la chute
      - angle_crit : angle limite au-delà duquel le torse est trop penché
    """

    # D'après la doc Walker2d-v5 :
    #   obs[0] = hauteur du torse
    #   obs[1] = angle du torse (0 = droit).  
    h = float(obs[0])
    angle = float(obs[1])

    # 1) Danger de hauteur : si le torse descend sous h_crit
    height_danger = max(0.0, h_crit - h)
    height_penalty = w_h * height_danger ** 2

    # 2) Danger d'angle : si |angle| dépasse angle_crit
    angle_danger = max(0.0, abs(angle) - angle_crit)
    angle_penalty = w_angle * angle_danger ** 2

    # Récompense finale : base - pénalités
    new_reward = float(base_reward) - height_penalty - angle_penalty
    return new_reward


In [21]:
import gymnasium as gym

class AntiFallProgressiveRewardWrapper(gym.Wrapper):
    def __init__(
        self,
        env,
        h_crit: float = 0.9,
        angle_crit: float = 0.5,
        w_h: float = 5.0,
        w_angle: float = 1.0,
    ):
        super().__init__(env)
        self.h_crit = h_crit
        self.angle_crit = angle_crit
        self.w_h = w_h
        self.w_angle = w_angle

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

    def step(self, action):
        # Env de base
        obs, base_reward, terminated, truncated, info = self.env.step(action)

        # Notre récompense "anti-chute progressive"
        new_reward = reward_anti_fall_progressive(
            obs=obs,
            action=action,
            info=info,
            base_reward=base_reward,
            h_crit=self.h_crit,
            angle_crit=self.angle_crit,
            w_h=self.w_h,
            w_angle=self.w_angle,
        )

        return obs, new_reward, terminated, truncated, info


In [22]:
from stable_baselines3 import SAC

# --- ENV D'ENTRAÎNEMENT (sans vidéo) ---

train_env_base = gym.make("Walker2d-v5")  # pas de render_mode ici

train_env = AntiFallProgressiveRewardWrapper(
    train_env_base,
    h_crit=0.9,
    angle_crit=0.5,
    w_h=5.0,
    w_angle=1.0,
)

model_anti_fall = SAC("MlpPolicy", train_env, verbose=1)
model_anti_fall.learn(total_timesteps=10_000)   # à augmenter plus tard

# optionnel : sauvegarder
# model_anti_fall.save("sac_walker2d_anti_fall_progressive")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16       |
|    ep_rew_mean     | -3.54    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 2133     |
|    time_elapsed    | 0        |
|    total_timesteps | 64       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16       |
|    ep_rew_mean     | -4.66    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 364      |
|    time_elapsed    | 0        |
|    total_timesteps | 128      |
| train/             |          |
|    actor_loss      | -6.49    |
|    critic_loss     | 2.8      |
|    ent_coef        | 0.992    |
|    ent_coef_loss   | -0.0795  |
|    learning_rate   | 0.0003   |
|    n_updates       | 27       |
----------------------

In [23]:
from gymnasium.wrappers import RecordVideo

video_folder = "./videos_anti_fall_progressive"

# --- ENV D'ÉVALUATION AVEC VIDÉO ---

eval_env_base = gym.make("Walker2d-v5", render_mode="rgb_array")

eval_env_wrapped = AntiFallProgressiveRewardWrapper(
    eval_env_base,
    h_crit=0.9,
    angle_crit=0.5,
    w_h=5.0,
    w_angle=1.0,
)

eval_env = RecordVideo(
    eval_env_wrapped,
    video_folder=video_folder,
    name_prefix="walker2d-anti_fall",
    episode_trigger=lambda ep_id: True,  # on filme le 1er épisode
    video_length=0,                       # épisode complet
)

# si tu as sauvegardé :
# model_anti_fall = SAC.load("sac_walker2d_anti_fall_progressive", env=eval_env)

obs, info = eval_env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action, _ = model_anti_fall.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)

eval_env.close()
print(f"Vidéo enregistrée dans : {video_folder}")


Vidéo enregistrée dans : ./videos_anti_fall_progressive


### Fonction de récompense 6 : anti-chute progressive (shaping autour des états dangereux)

Cette récompense a pour objectif d'aider le walker à **éviter la chute** en pénalisant progressivement les états "dangereux", au lieu d'avoir uniquement une grosse punition quand l'épisode se termine.

Dans `Walker2d-v5`, on utilise notamment :

- `base_reward` : la récompense de base de l'environnement (vitesse vers l'avant, survie, coût de contrôle)  
- `obs[0]` : la hauteur du torse  
- `obs[1]` : l'angle du torse (0 = torse vertical)

On introduit deux seuils :

- $h_{\text{crit}}$ : hauteur critique en dessous de laquelle le robot est considéré comme trop bas (proche de se coucher)  
- $\theta_{\text{crit}}$ : angle critique au-delà duquel le torse est trop penché

La récompense est définie comme :

$
r_t = \text{base\_reward}_t
      - w_h \, \max(0, h_{\text{crit}} - h_t)^2
      - w_{\text{angle}} \, \max(0, |\theta_t| - \theta_{\text{crit}})^2
$

où :

- $h_t$ est la hauteur du torse au temps t,
- $\theta_t$ est l'angle du torse,
- $w_h$ contrôle l'importance de la pénalisation de la hauteur dangereuse,
- $w_{\text{angle}}$ contrôle l'importance de la pénalisation de l'angle dangereux.

#### Intérêt de ce shaping

- Les états **proches de la chute** (trop bas, trop penchés) sont pénalisés avant la fin de l’épisode.  
- L’agent reçoit donc un **signal de danger progressif** qui lui permet d’apprendre à :
  - se redresser,
  - éviter de s’affaisser,
  - corriger sa posture au lieu de simplement tomber.

Dans un contexte de **bruit** (observations bruitées, resets aléatoires), cette récompense permet de tester si les algorithmes RL apprennent des comportements plus **robustes**, en réduisant :

- le nombre de chutes par épisode,
- le temps passé dans des postures dangereuses (hauteur < $h_{\text{crit}}$, angle > $\theta_{\text{crit}}$).


In [1]:
from typing import Dict, Any
import numpy as np

def reward_anti_fall_progressive(
    obs,
    action,
    info: Dict[str, Any],
    base_reward: float,
    h_crit: float = 0.9,
    angle_crit: float = 0.5,
    w_h: float = 5.0,
    w_angle: float = 1.0,
) -> float:
    """
    Récompense 6 : anti-chute progressive (shaping autour des états dangereux)

    Idée :
      - on part de la récompense de base de l'environnement (base_reward)
      - on enlève un bonus quand le walker se rapproche de la chute :
          * torse trop bas  (h < h_crit)
          * torse trop penché (|angle| > angle_crit)

    r_t = base_reward
          - w_h     * max(0, h_crit - h_t)^2
          - w_angle * max(0, |angle_t| - angle_crit)^2
    """

    # D'après Walker2d-v5 :
    #   obs[0] = hauteur du torse
    #   obs[1] = angle du torse (0 = vertical)
    h = float(obs[0])
    angle = float(obs[1])

    # Danger de hauteur : si le torse descend sous h_crit
    height_danger = max(0.0, h_crit - h)
    height_penalty = w_h * height_danger**2

    # Danger d'angle : si |angle| dépasse angle_crit
    angle_danger = max(0.0, abs(angle) - angle_crit)
    angle_penalty = w_angle * angle_danger**2

    new_reward = float(base_reward) - height_penalty - angle_penalty
    return new_reward


In [2]:
import gymnasium as gym

class AntiFallProgressiveRewardWrapper(gym.Wrapper):
    def __init__(
        self,
        env,
        h_crit: float = 0.9,
        angle_crit: float = 0.5,
        w_h: float = 5.0,
        w_angle: float = 1.0,
    ):
        super().__init__(env)
        self.h_crit = h_crit
        self.angle_crit = angle_crit
        self.w_h = w_h
        self.w_angle = w_angle

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

    def step(self, action):
        # step de l'env original
        obs, base_reward, terminated, truncated, info = self.env.step(action)

        # récompense 6 : anti-chute progressive
        new_reward = reward_anti_fall_progressive(
            obs=obs,
            action=action,
            info=info,
            base_reward=base_reward,
            h_crit=self.h_crit,
            angle_crit=self.angle_crit,
            w_h=self.w_h,
            w_angle=self.w_angle,
        )

        return obs, new_reward, terminated, truncated, info


In [3]:
from stable_baselines3 import SAC

# --- ENV D'ENTRAÎNEMENT (pas de vidéo ici) ---

train_env_base = gym.make("Walker2d-v5")  # pas de render_mode

train_env = AntiFallProgressiveRewardWrapper(
    train_env_base,
    h_crit=0.9,
    angle_crit=0.5,
    w_h=5.0,
    w_angle=1.0,
)

model_anti_fall = SAC("MlpPolicy", train_env, verbose=1)
model_anti_fall.learn(total_timesteps=10_000)  # à augmenter plus tard

# optionnel :
# model_anti_fall.save("sac_walker2d_anti_fall_progressive")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16.2     |
|    ep_rew_mean     | 0.436    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 1964     |
|    time_elapsed    | 0        |
|    total_timesteps | 65       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16.5     |
|    ep_rew_mean     | 1.11     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 269      |
|    time_elapsed    | 0        |
|    total_timesteps | 132      |
| train/             |          |
|    actor_loss      | -7.24    |
|    critic_loss     | 3.65     |
|    ent_coef        | 0.991    |
|    ent_coef_loss   | -0.0907  |
|    learning_rate   | 0.0003   |
|    n_updates       | 31       |
----------------------

In [4]:
from gymnasium.wrappers import RecordVideo

video_folder = "./videos_anti_fall_progressive"

# --- ENV D'ÉVALUATION AVEC VIDÉO ---

eval_env_base = gym.make("Walker2d-v5", render_mode="rgb_array")

eval_env_wrapped = AntiFallProgressiveRewardWrapper(
    eval_env_base,
    h_crit=0.9,
    angle_crit=0.5,
    w_h=5.0,
    w_angle=1.0,
)

eval_env = RecordVideo(
    eval_env_wrapped,
    video_folder=video_folder,
    name_prefix="walker2d-anti_fall",
    episode_trigger=lambda ep_id: True,
    video_length=0,  # épisode complet
)

# si tu avais sauvegardé :
# model_anti_fall = SAC.load("sac_walker2d_anti_fall_progressive", env=eval_env)

obs, info = eval_env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action, _ = model_anti_fall.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)

eval_env.close()
print(f"Vidéo enregistrée dans : {video_folder}")


c:\Users\arthu\miniconda3\envs\walker2d\lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at c:\Users\arthu\OneDrive\Documents\UTC\Poly mtl\RL\Projet\videos_anti_fall_progressive folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Vidéo enregistrée dans : ./videos_anti_fall_progressive


### Fonction de récompense 7 : marche robuste & économe (multi-objectif simple)

Cette récompense cherche à combiner explicitement trois objectifs :

1. **Aller vers l'avant** (vitesse),
2. **Consommer peu d'énergie** (actions modérées),
3. **Rester suffisamment haut** (posture globale stable).

Dans `Walker2d-v5` :

- `obs[8]` correspond à la vitesse en x du torse,
- `obs[0]` correspond à la hauteur du torse,
- l'énergie consommée peut être approximée par $\|a_t\|^2$ (norme au carré de l'action).

On définit :

$
r_t = w_v \, v_x
      - w_E \, \|a_t\|^2
      + w_h \, \max(0, h_t - h_{\min})
      + w_{\text{survive}} \cdot \text{reward\_survive}
$

où :

- $v_x$ est la vitesse horizontale du torse,
- $\|a_t\|^2$ mesure l'énergie instantanée des actions,
- $h_t$ est la hauteur du torse,
- $h_{\min}$ est une hauteur minimale souhaitée (par ex. 1.0),
- $w_v$ (`v_weight`) règle l’importance de la vitesse,
- $w_E$ (`energy_weight`) règle l’importance de l’économie d’énergie,
- $w_h$ (`h_weight`) règle l’importance de rester haut,
- $w_{\text{survive}}$ (`w_survive`) permet éventuellement de garder un terme de survie.

#### Intérêt pour la robustesse

Cette récompense permet d'étudier un compromis clair entre :

- **performance** (distance parcourue, vitesse moyenne),
- **coût énergétique** (somme des \(\|a_t\|^2\)),
- **stabilité globale** (hauteur moyenne du torse).

Sous perturbations (bruit d'observation, randomisation des resets), on peut comparer différentes politiques en observant :

- la distance parcourue,
- l'énergie totale consommée,
- la hauteur moyenne et le nombre de chutes.


In [7]:
from typing import Dict, Any
import numpy as np

def reward_robust_econ(
    obs,
    action,
    info: Dict[str, Any],
    v_weight: float = 1.0,
    energy_weight: float = 1e-3,
    h_weight: float = 1.0,
    h_min: float = 1.0,
    w_survive: float = 0.0,
) -> float:
    """
    Récompense 7 : marche robuste & économe (multi-objectif simple)

    Objectif : combiner trois critères :
      - vitesse vers l'avant (vx)
      - énergie consommée (||a_t||^2)
      - hauteur du torse (h)

    r_t = v_weight     * v_x
        - energy_weight * ||a_t||^2
        + h_weight     * max(0, h_t - h_min)
        + w_survive    * reward_survive

    où :
      - v_x     : vitesse en x du torse (obs[8] dans Walker2d-v5)
      - a_t     : action au temps t
      - h_t     : hauteur du torse (obs[0])
      - h_min   : hauteur minimale désirée
    """

    # Vitesse horizontale (gait plus rapide)
    vx = float(obs[8])          # velocity of x-coordinate of torso

    # Énergie des actions
    energy = float(np.sum(np.square(action)))

    # Hauteur du torse
    h = float(obs[0])

    # Terme de survie de l'env (optionnel)
    survive = float(info.get("reward_survive", 0.0))

    # Terme de hauteur : on récompense si h > h_min
    height_term = h_weight * max(0.0, h - h_min)

    reward = (
        v_weight * vx
        - energy_weight * energy
        + height_term
        + w_survive * survive
    )
    return reward


In [8]:
import gymnasium as gym

class RobustEconRewardWrapper(gym.Wrapper):
    def __init__(
        self,
        env,
        v_weight: float = 1.0,
        energy_weight: float = 1e-3,
        h_weight: float = 1.0,
        h_min: float = 1.0,
        w_survive: float = 0.0,
    ):
        super().__init__(env)
        self.v_weight = v_weight
        self.energy_weight = energy_weight
        self.h_weight = h_weight
        self.h_min = h_min
        self.w_survive = w_survive

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

    def step(self, action):
        # step de l'env de base
        obs, base_reward, terminated, truncated, info = self.env.step(action)

        # récompense 7 : marche robuste & économe
        new_reward = reward_robust_econ(
            obs=obs,
            action=action,
            info=info,
            v_weight=self.v_weight,
            energy_weight=self.energy_weight,
            h_weight=self.h_weight,
            h_min=self.h_min,
            w_survive=self.w_survive,
        )

        return obs, new_reward, terminated, truncated, info


In [9]:
from stable_baselines3 import SAC

# --- ENV D'ENTRAÎNEMENT (pas de vidéo ici) ---

train_env_base = gym.make("Walker2d-v5")

train_env = RobustEconRewardWrapper(
    train_env_base,
    v_weight=1.0,
    energy_weight=1e-3,
    h_weight=1.0,
    h_min=1.0,
    w_survive=0.0,  # tu peux tester 0.0 ou 1.0
)

model_robust_econ = SAC("MlpPolicy", train_env, verbose=1)
model_robust_econ.learn(total_timesteps=10_000)  # à augmenter plus tard

# optionnel :
# model_robust_econ.save("sac_walker2d_robust_econ")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.8     |
|    ep_rew_mean     | -17      |
| time/              |          |
|    episodes        | 4        |
|    fps             | 2079     |
|    time_elapsed    | 0        |
|    total_timesteps | 79       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.6     |
|    ep_rew_mean     | -14.3    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 220      |
|    time_elapsed    | 0        |
|    total_timesteps | 157      |
| train/             |          |
|    actor_loss      | -7.11    |
|    critic_loss     | 1.77     |
|    ent_coef        | 0.983    |
|    ent_coef_loss   | -0.167   |
|    learning_rate   | 0.0003   |
|    n_updates       | 56       |
----------------------

In [10]:
from gymnasium.wrappers import RecordVideo

video_folder = "./videos_robust_econ"

# --- ENV D'ÉVALUATION AVEC VIDÉO ---

eval_env_base = gym.make("Walker2d-v5", render_mode="rgb_array")

eval_env_wrapped = RobustEconRewardWrapper(
    eval_env_base,
    v_weight=1.0,
    energy_weight=1e-3,
    h_weight=1.0,
    h_min=1.0,
    w_survive=0.0,
)

eval_env = RecordVideo(
    eval_env_wrapped,
    video_folder=video_folder,
    name_prefix="walker2d-robust_econ",
    episode_trigger=lambda ep_id: True,  # on filme le 1er épisode
    video_length=0,                       # épisode complet
)

# si tu avais sauvegardé :
# model_robust_econ = SAC.load("sac_walker2d_robust_econ", env=eval_env)

obs, info = eval_env.reset()
terminated = False
truncated = False

while not (terminated or truncated):
    action, _ = model_robust_econ.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)

eval_env.close()
print(f"Vidéo enregistrée dans : {video_folder}")


Vidéo enregistrée dans : ./videos_robust_econ


les gregroupement : 

- Vitesse vs énergie : R1 ou R7
- Vitesse cible : R2
- Posture / éviter de tomber : R3 ou R6
- Actions lisses : R4